In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,BatchNormalization,Dropout,Flatten,Input
from tensorflow.keras import optimizers
from keras import Model
from tensorflow.keras import initializers
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.backend import clear_session

In [19]:
df = pd.read_csv('../input/dataset/train.csv')
df

In [ ]:
for i in df.columns[1:]:
    print(df[i].value_counts())

In [20]:
le_age = LabelEncoder()
le_race = LabelEncoder()
ages = le_age.fit_transform(df['age'])
races= le_race.fit_transform(df['race'])
df=pd.get_dummies(df, columns=['gender'], drop_first=True)

pair_age={idx:age for idx, age in enumerate(le_age.classes_)}
pair_race={idx:race for idx, race in enumerate(le_race.classes_)}
df['race_enc'] = races
df['age_enc'] = ages

df=df.sample(frac=1).reset_index()
pair_age, pair_race

In [21]:
y=df.iloc[:5000,-3:]
x=df['file'][:5000]

from sklearn.model_selection import train_test_split
x1,x2,y1,y2=train_test_split(x, y, test_size=0.35, random_state=47, shuffle=True)
# Here, 1 being the train data and 2 being the test data

x1.shape, x2.shape, y1.shape, y2.shape

In [ ]:
for i in y1.columns:
    print(y1[i].value_counts())

In [22]:
datagen = ImageDataGenerator(rotation_range=35, zoom_range=0.2, horizontal_flip=True,
                            brightness_range =(0.2,0.5),height_shift_range=0.2,
                             width_shift_range=0.2,rescale=1/255.)

folder_path = '../input/dataset/'
label_df = y1.reset_index(drop=True)
from tqdm import tqdm
images = np.empty((0,80,80,3))
labels = pd.DataFrame()
for i in tqdm(range(len(x1))):
    a=load_img(folder_path+x1.reset_index().file[i], target_size=(80,80))
    a=img_to_array(a)
    val = label_df['age_enc'][i] 
    flow = datagen.flow(a.reshape((1,80,80,3)), batch_size=1)
    if val== 1 or val==6:
        for j in range(2):
            aug = flow.next()
            images = np.append(images, aug, axis=0)
            labels = pd.concat([labels, label_df.iloc[i:i+1,:]])
            
    if val==7:
        for j in range(6):
            aug = flow.next()
            images = np.append(images, aug, axis=0)
            labels = pd.concat([labels, label_df.iloc[i:i+1,:]])
                       
    if  val==0 or val==8:
        for j in range(10):
            aug = flow.next()
            images = np.append(images, aug, axis=0)
            labels = pd.concat([labels, label_df.iloc[i:i+1,:]])
    else:
        a=a/255.
        images = np.append(images, a.reshape([1,80,80,3]), axis=0)
        labels = pd.concat([labels, label_df.iloc[i:i+1,:]])


In [23]:
from sklearn.utils import shuffle
images,labels = shuffle(images, np.array(labels), random_state=59)

In [67]:
ran = np.random.randint(0,len(images),1)
print(ran)
vals =labels[ran].flatten()

plt.imshow(images[ran][0])
if vals[0] ==1:
    g='Male'
else:g='Female'
plt.title(g + pair_race[vals[1]] + pair_age[vals[2]])

In [24]:
test_images=[]
for file in tqdm(x2):
    img = load_img(folder_path + file, target_size=(80,80))
    img = img_to_array(img)/255.
    test_images.append(img)

test_images = np.array(test_images)

In [25]:
from tensorflow.keras.applications import InceptionResNetV2
conv_base4 = InceptionResNetV2(include_top=False,weights="imagenet",
                               input_shape=(80,80,3))

In [ ]:
import tensorflow as tf
tf.random.set_seed(23)

In [ ]:
!pip install focal_loss

In [41]:
from focal_loss import SparseCategoricalFocalLoss

In [68]:
input_layer = Input(shape=(80,80,3))
first_dense=conv_base4(input_layer)
first_dense=Flatten()(first_dense)
first_dense=Dropout(0.35)(first_dense)
#first_dense=(0.5)(first_dense)
first_dense = Dense(500, activation='relu')(first_dense)
first_dense = BatchNormalization()(first_dense)

f1=Dense(200, activation='relu')(first_dense)
f1=Dropout(0.35)(f1)
f1=BatchNormalization()(f1)
f2=Dense(200, activation='relu')(f1)
f1=Dropout(0.35)(f2)
f1=BatchNormalization()(f2)
y1_output = Dense(1, activation='sigmoid', name='gender')(f2)

second_dense = Dense(500,activation='relu')(first_dense)
second_dense = Dropout(0.35)(second_dense)
second_dense = BatchNormalization()(second_dense)
second_dense = Dense(300,activation='relu')(second_dense)
second_dense = Dropout(0.35)(second_dense)
y2_output = Dense(7, activation='softmax', name='race')(second_dense)

third_dense = Dense(500, activation='relu')(first_dense)
third_dense = Dropout(0.35)(third_dense)
third_dense = BatchNormalization()(third_dense)
third_dense = Dense(300, activation='relu')(third_dense)
third_dense = Dropout(0.35)(third_dense)
third_dense = BatchNormalization()(third_dense)
y3_output = Dense(9, activation='softmax', name='age')(third_dense)

model = Model(inputs=input_layer,outputs=[y1_output, y2_output, y3_output])
conv_base4.trainable = False

optimizer =optimizers.Adam(lr=0.00013)
model.compile(optimizer=optimizer,
            loss={'gender':'binary_crossentropy',
                  'race':SparseCategoricalFocalLoss(gamma=2.5),
                  'age': SparseCategoricalFocalLoss(gamma=2.5)},metrics=['accuracy'])
model.summary()

In [69]:
g1=labels[:,0]
r1=labels[:,1]
a1=labels[:,2]
w1=g1,r1,a1

g2=np.array(y2['gender_Male'])
r2=np.array(y2['race_enc'])
a2=np.array(y2['age_enc'])
w2 = g2, r2, a2

In [70]:
his = model.fit(x=images,y=w1,epochs=30, batch_size=28,
         validation_data=(test_images,w2))

In [72]:
model.save_weights('face1.h5')

In [71]:
model.save('face.h5')

In [45]:
from tensorflow.keras.backend import clear_session
del model
clear_session()